In [1]:
%%time
import json
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import first

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

CPU times: user 115 ms, sys: 31.7 ms, total: 147 ms
Wall time: 20.8 s


In [ ]:
## de  ultracom/compensar/tasks/constants.py 

cte_PRIMARY_KEYS = {
            'flowConnection': ["name"],
            'flowVariable': ["nameSpace", "name"],
            'flowParameter': ["nameSpace", "parameterName"],
            'flowTask': ["name"],
            'taskOutput': ["key"],
            'connectionType': ["type"]
        }

In [4]:
%%bash

ls

Untitled.ipynb
response_1600883647838.json


In [ ]:
with open('response_1600883647838.json') as f:
    flowInfo = json.load(f)

#keys = [el for el in list(flowInfo.keys()) if type(flowInfo[el]) is list]
keys = filter(lambda key: isinstance(flowInfo[key],list), flowInfo.keys())

In [2]:
%%time
df = spark.read.option("multiline","true").json("response_1600883647838.json")

CPU times: user 5.81 ms, sys: 3.3 ms, total: 9.11 ms
Wall time: 18.9 s


In [3]:
%%time
df.show()

+--------------+--------------------+--------------------+--------------------+---+--------------------+---------------+
|flowConnection|       flowParameter|            flowTask|        flowVariable| id|          taskOutput|        version|
+--------------+--------------------+--------------------+--------------------+---+--------------------+---------------+
|            []|[[global, exprexi...|[[, [[[{taskOutpu...|[[String, -, 1, f...|  1|[[1, messageSucce...|[0, 1, 1, 0, 0]|
+--------------+--------------------+--------------------+--------------------+---+--------------------+---------------+

CPU times: user 3.97 ms, sys: 2.2 ms, total: 6.17 ms
Wall time: 14 s


In [ ]:
@F.udf(returnType=BooleanType())
def get_column_with_list(colFirstValue):
    return isinstance(colFirstValue, list)


In [ ]:
%%time
# https://stackoverflow.com/questions/46813283/select-columns-in-pyspark-dataframe
df_linea89 = df.select(keys)

In [ ]:
%%time
df_linea89.show()

In [ ]:
%%time
##linea 93
## https://docs.databricks.com/spark/latest/spark-sql/udf-python.html

@F.udf("string")
def getName(column_name,listOfItem):
    if len(cte_PRIMARY_KEYS[column_name]) > 1:
        pk = "_".join([listOfItem[i] for i in range(len(cte_PRIMARY_KEYS[column_name]))])
    else:
        pk = listOfItem[0]
    return pk

In [ ]:
%%time
# https://stackoverflow.com/questions/34037889/apply-same-function-to-all-fields-of-spark-dataframe-row
# https://stackoverflow.com/questions/36584812/pyspark-row-wise-function-composition
# df_linea101_pk = df_linea89.select(*(getName(c.name,F.col(c)) for c in df_linea89.columns))
df_linea101_pk = df_linea89.select(get_name(struct([x.name,df_linea89[x], for x in df_linea89.columns])))

In [ ]:
%%time
## la linea 106 y la 101 hacen lo mismo
# df_linea89.withColumn("null_count", count_empty_columns(struct([df[x] for x in df_linea89.columns])))

In [ ]:
%%time
# https://stackoverflow.com/questions/44067861/pyspark-add-a-new-column-with-a-tuple-created-from-columns
df_linea109_namedTuples = df_linea89.withColumn("namedTuples",\
                                                struct([get_name(x.name,df_linea89[x])],\
                                                        x.name for x in df_linea89.columns))

In [ ]:
%%time
# https://stackoverflow.com/questions/47682927/how-to-create-a-udf-in-pyspark-which-returns-an-array-of-strings
get_values_in_ = udf(my_udf, ArrayType(StringType()))

In [ ]:
#

df_linea109_namedTuplesPredefined.withColumn("namedTuplesPredefined", x for x in df_linea89.columns)